In [1]:
import os
import cv2
import numpy as np
import h5py
from matplotlib import pyplot as plt
import keras
from keras.models import Model, Sequential
from keras.applications.vgg16 import VGG16
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras import layers
from keras.layers import *
from sklearn.utils import shuffle
from PIL import Image

In [2]:
CKdata = h5py.File('C:/Users/Student/Desktop/Facial_Emotion_Detection/Ck+/Ck_data.h5', 'r', driver='core')

In [3]:
X_data = np.expand_dims(np.asarray(CKdata['data_pixel']), axis=-1)

In [4]:
Y_data = np.asarray(CKdata['data_label'])

In [5]:
Y_data = to_categorical(Y_data)

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size=0.2)

In [7]:
num_train = X_train.shape[0]

In [8]:
num_val = X_test.shape[0]

In [9]:
print('Training: ',X_train.shape)

Training:  (784, 48, 48, 1)


In [10]:
print('Validation: ',X_test.shape)

Validation:  (197, 48, 48, 1)


In [11]:
from keras.layers import Convolution2D, BatchNormalization, Activation, Dropout, Dense, Flatten, MaxPooling2D, AveragePooling2D

In [12]:
from keras.models import Sequential

In [13]:
def get_cnn_model():
    model = Sequential()

    model.add(Convolution2D(64, (3, 1), padding='same', input_shape=(48,48,1)))
    model.add(Convolution2D(64, (1, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='same'))
    model.add(Dropout(0.25))

    model.add(Convolution2D(128, (3, 1), padding='same'))
    model.add(Convolution2D(128, (1, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='same'))
    model.add(Dropout(0.25))

    model.add(Convolution2D(256, (3, 1), padding='same'))
    model.add(Convolution2D(256, (1, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='same'))
    model.add(Dropout(0.25))

    model.add(Convolution2D(512, (3, 1), padding='same'))
    model.add(Convolution2D(512, (1, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='same'))
    model.add(Dropout(0.25))

    model.add(Flatten())

    model.add(Dense(512))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.25))

    model.add(Dense(256))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.25))

    model.add(Dense(7))
    model.add(Activation('softmax'))
    return model

In [14]:
cnn_model = get_cnn_model()

In [15]:
cnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 64)        256       
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 48, 64)        12352     
                                                                 
 batch_normalization (BatchN  (None, 48, 48, 64)       256       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 48, 48, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 24, 24, 64)        0

In [16]:
from keras.preprocessing.image import ImageDataGenerator 

In [17]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [18]:
from keras import backend as K

In [19]:
def fbeta(y_true, y_pred, threshold_shift=0):
    beta = 1
    y_pred = K.clip(y_pred, 0, 1)
    y_pred_bin = K.round(y_pred + threshold_shift)

    tp = K.sum(K.round(y_true * y_pred_bin), axis=1) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)), axis=1)
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)), axis=1)

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    beta_squared = beta ** 2
    return K.mean((beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon()))

In [20]:
def train_cnn_model(model):
    filepath='../opt/ck-cnn/Model.{epoch:02d}-{val_acc:.4f}.hdf5'
    checkpointer = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, mode='auto')
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=0, mode='auto', cooldown=0, min_lr=0)
    early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto')
    datagen = ImageDataGenerator(
        featurewise_center=False,  
        samplewise_center=False,  
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        zca_whitening=False,  
        rotation_range=0,  
        width_shift_range=0.0,  
        height_shift_range=0.0,  
        horizontal_flip=True,  
        vertical_flip=False,   
        )


    datagen.fit(X_train)
    datagen.fit(X_test)    
    batch_size = 32

    num_epochs = 200
    model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=[fbeta, 'acc'])
    train_flow = datagen.flow(X_train, Y_train, batch_size=batch_size)
    test_flow = datagen.flow(X_test, Y_test)
    history = model.fit_generator(train_flow,
                    steps_per_epoch=len(X_train) / batch_size,
                    epochs=num_epochs, 
                    verbose=1, 
                    validation_data=test_flow, 
                    validation_steps=len(X_test) / batch_size,
                    callbacks=[checkpointer, reduce_lr, checkpointer])
    return history, model

In [21]:
history, cnn_model = train_cnn_model(cnn_model)

Epoch 1/200


C:\Users\Student\AppData\Local\Temp\ipykernel_6848\1285611036.py:30: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_flow,


25/24 [==============================] - ETA: 0s - loss: 1.6820 - fbeta: 0.2438 - acc: 0.4273
Epoch 1: saving model to ../opt/ck-cnn\Model.01-0.2741.hdf5

Epoch 1: saving model to ../opt/ck-cnn\Model.01-0.2741.hdf5
24/24 [==============================] - 16s 567ms/step - loss: 1.6820 - fbeta: 0.2438 - acc: 0.4273 - val_loss: 6.2504 - val_fbeta: 0.2563 - val_acc: 0.2741 - lr: 0.0010
Epoch 2/200
25/24 [==============================] - ETA: 0s - loss: 1.0048 - fbeta: 0.5288 - acc: 0.6429
Epoch 2: saving model to ../opt/ck-cnn\Model.02-0.4975.hdf5

Epoch 2: saving model to ../opt/ck-cnn\Model.02-0.4975.hdf5
24/24 [==============================] - 14s 560ms/step - loss: 1.0048 - fbeta: 0.5288 - acc: 0.6429 - val_loss: 4.3913 - val_fbeta: 0.5054 - val_acc: 0.4975 - lr: 0.0010
Epoch 3/200
25/24 [==============================] - ETA: 0s - loss: 0.7492 - fbeta: 0.6313 - acc: 0.7232
Epoch 3: saving model to ../opt/ck-cnn\Model.03-0.6599.hdf5

Epoch 3: saving model to ../opt/ck-cnn\Model.03-0

24/24 [==============================] - 13s 551ms/step - loss: 0.1123 - fbeta: 0.9600 - acc: 0.9656 - val_loss: 0.1221 - val_fbeta: 0.9464 - val_acc: 0.9391 - lr: 0.0010
Epoch 22/200
25/24 [==============================] - ETA: 0s - loss: 0.0957 - fbeta: 0.9588 - acc: 0.9656
Epoch 22: saving model to ../opt/ck-cnn\Model.22-0.9695.hdf5

Epoch 22: saving model to ../opt/ck-cnn\Model.22-0.9695.hdf5
24/24 [==============================] - 13s 515ms/step - loss: 0.0957 - fbeta: 0.9588 - acc: 0.9656 - val_loss: 0.0754 - val_fbeta: 0.9732 - val_acc: 0.9695 - lr: 0.0010
Epoch 23/200
25/24 [==============================] - ETA: 0s - loss: 0.0740 - fbeta: 0.9700 - acc: 0.9758
Epoch 23: saving model to ../opt/ck-cnn\Model.23-0.7614.hdf5

Epoch 23: saving model to ../opt/ck-cnn\Model.23-0.7614.hdf5
24/24 [==============================] - 12s 495ms/step - loss: 0.0740 - fbeta: 0.9700 - acc: 0.9758 - val_loss: 0.6919 - val_fbeta: 0.7857 - val_acc: 0.7614 - lr: 0.0010
Epoch 24/200
25/24 [=======

Epoch 42/200
25/24 [==============================] - ETA: 0s - loss: 0.0305 - fbeta: 0.9875 - acc: 0.9898
Epoch 42: saving model to ../opt/ck-cnn\Model.42-0.9949.hdf5

Epoch 42: saving model to ../opt/ck-cnn\Model.42-0.9949.hdf5
24/24 [==============================] - 13s 538ms/step - loss: 0.0305 - fbeta: 0.9875 - acc: 0.9898 - val_loss: 0.0237 - val_fbeta: 0.9714 - val_acc: 0.9949 - lr: 1.0000e-05
Epoch 43/200
25/24 [==============================] - ETA: 0s - loss: 0.0213 - fbeta: 0.9925 - acc: 0.9923
Epoch 43: saving model to ../opt/ck-cnn\Model.43-0.9949.hdf5

Epoch 43: saving model to ../opt/ck-cnn\Model.43-0.9949.hdf5
24/24 [==============================] - 13s 517ms/step - loss: 0.0213 - fbeta: 0.9925 - acc: 0.9923 - val_loss: 0.0227 - val_fbeta: 0.9955 - val_acc: 0.9949 - lr: 1.0000e-05
Epoch 44/200
25/24 [==============================] - ETA: 0s - loss: 0.0162 - fbeta: 0.9950 - acc: 0.9974
Epoch 44: saving model to ../opt/ck-cnn\Model.44-0.9949.hdf5

Epoch 44: saving mode

...

25/24 [==============================] - ETA: 0s - loss: 0.0180 - fbeta: 0.9937 - acc: 0.9962
Epoch 122: saving model to ../opt/ck-cnn\Model.122-0.9949.hdf5

Epoch 122: saving model to ../opt/ck-cnn\Model.122-0.9949.hdf5
24/24 [==============================] - 13s 515ms/step - loss: 0.0180 - fbeta: 0.9937 - acc: 0.9962 - val_loss: 0.0179 - val_fbeta: 0.9955 - val_acc: 0.9949 - lr: 1.0000e-13
Epoch 123/200
25/24 [==============================] - ETA: 0s - loss: 0.0257 - fbeta: 0.9900 - acc: 0.9923
Epoch 123: saving model to ../opt/ck-cnn\Model.123-0.9949.hdf5

Epoch 123: saving model to ../opt/ck-cnn\Model.123-0.9949.hdf5
24/24 [==============================] - 13s 522ms/step - loss: 0.0257 - fbeta: 0.9900 - acc: 0.9923 - val_loss: 0.0115 - val_fbeta: 0.9955 - val_acc: 0.9949 - lr: 1.0000e-13
Epoch 124/200
25/24 [==============================] - ETA: 0s - loss: 0.0198 - fbeta: 0.9937 - acc: 0.9949
Epoch 124: saving model to ../opt/ck-cnn\Model.124-0.9949.hdf5

Epoch 124: saving mode

25/24 [==============================] - ETA: 0s - loss: 0.0170 - fbeta: 0.9962 - acc: 0.9962
Epoch 142: saving model to ../opt/ck-cnn\Model.142-0.9949.hdf5

Epoch 142: saving model to ../opt/ck-cnn\Model.142-0.9949.hdf5
24/24 [==============================] - 14s 557ms/step - loss: 0.0170 - fbeta: 0.9962 - acc: 0.9962 - val_loss: 0.0187 - val_fbeta: 0.9955 - val_acc: 0.9949 - lr: 1.0000e-15
Epoch 143/200
25/24 [==============================] - ETA: 0s - loss: 0.0194 - fbeta: 0.9925 - acc: 0.9962
Epoch 143: saving model to ../opt/ck-cnn\Model.143-0.9949.hdf5

Epoch 143: saving model to ../opt/ck-cnn\Model.143-0.9949.hdf5
24/24 [==============================] - 13s 550ms/step - loss: 0.0194 - fbeta: 0.9925 - acc: 0.9962 - val_loss: 0.0121 - val_fbeta: 0.9955 - val_acc: 0.9949 - lr: 1.0000e-15
Epoch 144/200
25/24 [==============================] - ETA: 0s - loss: 0.0118 - fbeta: 0.9987 - acc: 0.9987
Epoch 144: saving model to ../opt/ck-cnn\Model.144-0.9949.hdf5

Epoch 144: saving mode

25/24 [==============================] - ETA: 0s - loss: 0.0247 - fbeta: 0.9925 - acc: 0.9936
Epoch 162: saving model to ../opt/ck-cnn\Model.162-0.9949.hdf5

Epoch 162: saving model to ../opt/ck-cnn\Model.162-0.9949.hdf5
24/24 [==============================] - 13s 536ms/step - loss: 0.0247 - fbeta: 0.9925 - acc: 0.9936 - val_loss: 0.0117 - val_fbeta: 0.9955 - val_acc: 0.9949 - lr: 1.0000e-17
Epoch 163/200
25/24 [==============================] - ETA: 0s - loss: 0.0177 - fbeta: 0.9962 - acc: 0.9962
Epoch 163: saving model to ../opt/ck-cnn\Model.163-0.9949.hdf5

Epoch 163: saving model to ../opt/ck-cnn\Model.163-0.9949.hdf5
24/24 [==============================] - 13s 518ms/step - loss: 0.0177 - fbeta: 0.9962 - acc: 0.9962 - val_loss: 0.0119 - val_fbeta: 0.9955 - val_acc: 0.9949 - lr: 1.0000e-17
Epoch 164/200
25/24 [==============================] - ETA: 0s - loss: 0.0247 - fbeta: 0.9937 - acc: 0.9949
Epoch 164: saving model to ../opt/ck-cnn\Model.164-0.9949.hdf5

Epoch 164: saving mode

25/24 [==============================] - ETA: 0s - loss: 0.0235 - fbeta: 0.9925 - acc: 0.9949
Epoch 182: saving model to ../opt/ck-cnn\Model.182-0.9949.hdf5

Epoch 182: saving model to ../opt/ck-cnn\Model.182-0.9949.hdf5
24/24 [==============================] - 14s 557ms/step - loss: 0.0235 - fbeta: 0.9925 - acc: 0.9949 - val_loss: 0.0193 - val_fbeta: 0.9955 - val_acc: 0.9949 - lr: 1.0000e-19
Epoch 183/200
25/24 [==============================] - ETA: 0s - loss: 0.0236 - fbeta: 0.9937 - acc: 0.9936
Epoch 183: saving model to ../opt/ck-cnn\Model.183-0.9949.hdf5

Epoch 183: saving model to ../opt/ck-cnn\Model.183-0.9949.hdf5
24/24 [==============================] - 14s 561ms/step - loss: 0.0236 - fbeta: 0.9937 - acc: 0.9936 - val_loss: 0.0122 - val_fbeta: 0.9955 - val_acc: 0.9949 - lr: 1.0000e-19
Epoch 184/200
25/24 [==============================] - ETA: 0s - loss: 0.0218 - fbeta: 0.9925 - acc: 0.9936
Epoch 184: saving model to ../opt/ck-cnn\Model.184-0.9949.hdf5

Epoch 184: saving mode

RuntimeError: Can't decrement id ref count (unable to extend file properly)

In [ ]:
n_classes = Y_data.shape[1]

In [ ]:
labels=range(n_classes)

In [ ]:
def eval_model(history, model):
    batch_size = 32
    score = model.evaluate(X_test, Y_test, steps=(int)(len(X_test) / batch_size))
    print('Evaluation loss: ', score[0])
    print('Evaluation accuracy: ', score[1])
    # summarize history for accuracy
    plt.plot(history.history['acc'], color='b', label='Training')
    plt.plot(history.history['val_acc'], color='g', label='Validation')
    plt.title('Accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(loc='upper left')
    plt.show()

    # summarize history for loss
    plt.plot(history.history['loss'], color='b', label='Training')
    plt.plot(history.history['val_loss'], color='g', label='Validation')
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(loc='lower left')
    plt.show()
    
    

   

In [ ]:
eval_model(history, cnn_model)